In [1]:
from collections import defaultdict, Counter
import concurrent.futures
from functools import reduce
import glob

import pandas as pd
import simplejson as json
import jieba

In [2]:
words_filename = '../words/words.csv'
jieba_dict_filename = '../words/dict.txt.big'
jieba.set_dictionary(jieba_dict_filename)

In [3]:
def wordlist_to_wordset(word_list):
    words = []
    for word_group in word_list:
        if type(word_group) is str:
            words.extend([c.strip() for c in word_group.split('/')])
    return set(words)

def build_tw_cn_dict(tw_word_list, cn_word_list):
    
    tw2cn = defaultdict(list)
    cn2tw = defaultdict(list)
    
    for tw_word_group, cn_word_group in zip(tw_word_list, cn_word_list):
        if type(tw_word_group) is str:
            tw_words = [c.strip() for c in tw_word_group.split('/')]
        else:
            tw_words = [None]
        
        if type(cn_word_group) is str:
            cn_words = [c.strip() for c in cn_word_group.split('/')]
        else:
            tw_words = [None]
            
        for tw_word in tw_words:
            for cn_word in cn_words:
                tw2cn[tw_word].append(cn_word)
                cn2tw[cn_word].append(tw_word)
    
    return tw2cn, cn2tw

def new_defined_wordset(tw_word_list, cn_word_list):
    
    wordset = set()
    for tw_word_group, cn_word_group in zip(tw_word_list, cn_word_list):
        if type(tw_word_group) is str and type(cn_word_group) is str:
            tw_words = [c.strip() for c in tw_word_group.split('/')]
            cn_words = set([c.strip() for c in cn_word_group.split('/')])
            for cn_word in cn_words:
                for tw_word in tw_words:
                    if cn_word in tw_word:
                        break
                else:
                    wordset.add(cn_word)
    
    wordset -= wordlist_to_wordset(tw_word_list)
    return wordset

In [4]:
df = pd.read_csv(words_filename)
cn_word_list = df['cn_word'].tolist()
tw_word_list = df['tw_word'].tolist()
cn_word_set = wordlist_to_wordset(cn_word_list)
tw_word_set = wordlist_to_wordset(tw_word_list)
word_set = cn_word_set - tw_word_set
# filtered_word_set = new_defined_wordset(tw_word_list, cn_word_list)
tw2cn, cn2tw = build_tw_cn_dict(tw_word_list, cn_word_list)

In [5]:
def find_cn_words(article):
    cn_word_count = Counter()
    seg_list = jieba.cut(article, cut_all=False)
    seg_set = set(seg_list)
    
    return seg_set & word_set

In [6]:
def find_cn_words_from_filename(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        article = json.load(f)
        result = find_cn_words(article['content'])
        if result:
            print(article['url'], result)
        return Counter(result)

In [12]:
import time

freq_counter = Counter()
start_time = time.time()
for idx, filename in enumerate(filenames, 1):
    freq_counter += find_cn_words_from_filename(filename)
    if idx % 100 == 0:
        print('Has completed %d jobs' % idx)
        print('Time elapsed: %f sec' % (time.time() - start_time))

Has completed 100 jobs
Time elapsed: 1.684376 sec
Has completed 200 jobs
Time elapsed: 3.623185 sec
Has completed 300 jobs
Time elapsed: 5.197073 sec
Has completed 400 jobs
Time elapsed: 6.804704 sec
Has completed 500 jobs
Time elapsed: 9.058501 sec
Has completed 600 jobs
Time elapsed: 11.029980 sec
Has completed 700 jobs
Time elapsed: 13.095075 sec
Has completed 800 jobs
Time elapsed: 15.066096 sec
Has completed 900 jobs
Time elapsed: 16.966455 sec
Has completed 1000 jobs
Time elapsed: 18.621367 sec
Has completed 1100 jobs
Time elapsed: 20.741680 sec
Has completed 1200 jobs
Time elapsed: 22.569165 sec
Has completed 1300 jobs
Time elapsed: 24.505848 sec
Has completed 1400 jobs
Time elapsed: 26.566295 sec
Has completed 1500 jobs
Time elapsed: 28.433637 sec
Has completed 1600 jobs
Time elapsed: 30.520096 sec
Has completed 1700 jobs
Time elapsed: 32.467821 sec
Has completed 1800 jobs
Time elapsed: 34.235077 sec
Has completed 1900 jobs
Time elapsed: 35.777641 sec
Has completed 2000 jobs
Ti

Has completed 15600 jobs
Time elapsed: 289.948839 sec
Has completed 15700 jobs
Time elapsed: 291.833802 sec
Has completed 15800 jobs
Time elapsed: 293.934811 sec
Has completed 15900 jobs
Time elapsed: 295.603940 sec
Has completed 16000 jobs
Time elapsed: 297.726820 sec
Has completed 16100 jobs
Time elapsed: 299.325246 sec
Has completed 16200 jobs
Time elapsed: 301.258444 sec
Has completed 16300 jobs
Time elapsed: 303.011214 sec
Has completed 16400 jobs
Time elapsed: 304.890756 sec
Has completed 16500 jobs
Time elapsed: 306.724864 sec
Has completed 16600 jobs
Time elapsed: 308.628384 sec
Has completed 16700 jobs
Time elapsed: 310.471155 sec
Has completed 16800 jobs
Time elapsed: 312.358535 sec
Has completed 16900 jobs
Time elapsed: 314.367806 sec
Has completed 17000 jobs
Time elapsed: 316.003733 sec
Has completed 17100 jobs
Time elapsed: 318.004829 sec
Has completed 17200 jobs
Time elapsed: 319.737660 sec
Has completed 17300 jobs
Time elapsed: 321.643143 sec
Has completed 17400 jobs
Tim

Has completed 27100 jobs
Time elapsed: 484.750608 sec
Has completed 27200 jobs
Time elapsed: 486.728299 sec
Has completed 27300 jobs
Time elapsed: 488.395028 sec
Has completed 27400 jobs
Time elapsed: 489.716495 sec
Has completed 27500 jobs
Time elapsed: 491.253846 sec
Has completed 27600 jobs
Time elapsed: 492.951571 sec
Has completed 27700 jobs
Time elapsed: 495.587421 sec
Has completed 27800 jobs
Time elapsed: 497.716278 sec
Has completed 27900 jobs
Time elapsed: 499.323168 sec
Has completed 28000 jobs
Time elapsed: 501.271103 sec
Has completed 28100 jobs
Time elapsed: 503.173278 sec
Has completed 28200 jobs
Time elapsed: 504.893394 sec
Has completed 28300 jobs
Time elapsed: 506.077449 sec
Has completed 28400 jobs
Time elapsed: 507.529485 sec
Has completed 28500 jobs
Time elapsed: 509.202990 sec
Has completed 28600 jobs
Time elapsed: 510.715796 sec
Has completed 28700 jobs
Time elapsed: 512.790744 sec
Has completed 28800 jobs
Time elapsed: 514.511575 sec
Has completed 28900 jobs
Tim

Has completed 42300 jobs
Time elapsed: 743.908774 sec
Has completed 42400 jobs
Time elapsed: 745.759018 sec
Has completed 42500 jobs
Time elapsed: 747.364099 sec
Has completed 42600 jobs
Time elapsed: 749.314874 sec
Has completed 42700 jobs
Time elapsed: 750.865845 sec
Has completed 42800 jobs
Time elapsed: 752.300901 sec
Has completed 42900 jobs
Time elapsed: 753.898443 sec
Has completed 43000 jobs
Time elapsed: 755.182628 sec
Has completed 43100 jobs
Time elapsed: 756.504640 sec
Has completed 43200 jobs
Time elapsed: 757.482067 sec
Has completed 43300 jobs
Time elapsed: 759.382347 sec
Has completed 43400 jobs
Time elapsed: 761.016864 sec
Has completed 43500 jobs
Time elapsed: 762.562978 sec
Has completed 43600 jobs
Time elapsed: 764.225237 sec
Has completed 43700 jobs
Time elapsed: 765.771676 sec
Has completed 43800 jobs
Time elapsed: 767.346103 sec
Has completed 43900 jobs
Time elapsed: 769.078698 sec
Has completed 44000 jobs
Time elapsed: 770.532812 sec
Has completed 44100 jobs
Tim

Has completed 57500 jobs
Time elapsed: 999.937946 sec
Has completed 57600 jobs
Time elapsed: 1001.533110 sec
Has completed 57700 jobs
Time elapsed: 1003.507923 sec
Has completed 57800 jobs
Time elapsed: 1005.318292 sec
Has completed 57900 jobs
Time elapsed: 1007.063025 sec
Has completed 58000 jobs
Time elapsed: 1008.531389 sec
Has completed 58100 jobs
Time elapsed: 1010.083422 sec
Has completed 58200 jobs
Time elapsed: 1011.828606 sec
Has completed 58300 jobs
Time elapsed: 1013.299890 sec
Has completed 58400 jobs
Time elapsed: 1014.753269 sec
Has completed 58500 jobs
Time elapsed: 1015.905819 sec
Has completed 58600 jobs
Time elapsed: 1017.596661 sec
Has completed 58700 jobs
Time elapsed: 1019.397997 sec
Has completed 58800 jobs
Time elapsed: 1021.247080 sec
Has completed 58900 jobs
Time elapsed: 1022.688424 sec
Has completed 59000 jobs
Time elapsed: 1024.960100 sec
Has completed 59100 jobs
Time elapsed: 1026.310238 sec
Has completed 59200 jobs
Time elapsed: 1027.797209 sec
Has complet

...

Has completed 354000 jobs
Time elapsed: 3350.869797 sec
Has completed 354100 jobs
Time elapsed: 3351.164148 sec
Has completed 354200 jobs
Time elapsed: 3351.473239 sec
Has completed 354300 jobs
Time elapsed: 3351.774334 sec
Has completed 354400 jobs
Time elapsed: 3352.070877 sec
Has completed 354500 jobs
Time elapsed: 3352.371608 sec
Has completed 354600 jobs
Time elapsed: 3352.656851 sec
Has completed 354700 jobs
Time elapsed: 3352.935796 sec
Has completed 354800 jobs
Time elapsed: 3353.250641 sec
Has completed 354900 jobs
Time elapsed: 3353.529873 sec
Has completed 355000 jobs
Time elapsed: 3353.799743 sec
Has completed 355100 jobs
Time elapsed: 3354.071249 sec
Has completed 355200 jobs
Time elapsed: 3354.339792 sec
Has completed 355300 jobs
Time elapsed: 3354.628224 sec
Has completed 355400 jobs
Time elapsed: 3354.959578 sec
Has completed 355500 jobs
Time elapsed: 3355.245309 sec
Has completed 355600 jobs
Time elapsed: 3355.605675 sec
Has completed 355700 jobs
Time elapsed: 3355.926

Has completed 368700 jobs
Time elapsed: 3396.721452 sec
Has completed 368800 jobs
Time elapsed: 3396.975191 sec
Has completed 368900 jobs
Time elapsed: 3397.344520 sec
Has completed 369000 jobs
Time elapsed: 3397.736910 sec
Has completed 369100 jobs
Time elapsed: 3398.039331 sec
Has completed 369200 jobs
Time elapsed: 3398.345569 sec
Has completed 369300 jobs
Time elapsed: 3398.617903 sec
Has completed 369400 jobs
Time elapsed: 3398.886814 sec
Has completed 369500 jobs
Time elapsed: 3399.150032 sec
Has completed 369600 jobs
Time elapsed: 3399.443807 sec
Has completed 369700 jobs
Time elapsed: 3399.746034 sec
Has completed 369800 jobs
Time elapsed: 3400.054325 sec
Has completed 369900 jobs
Time elapsed: 3400.347867 sec
Has completed 370000 jobs
Time elapsed: 3400.641322 sec
Has completed 370100 jobs
Time elapsed: 3400.936399 sec
Has completed 370200 jobs
Time elapsed: 3401.253655 sec
Has completed 370300 jobs
Time elapsed: 3401.554117 sec
Has completed 370400 jobs
Time elapsed: 3401.864

Has completed 383400 jobs
Time elapsed: 3441.316554 sec
Has completed 383500 jobs
Time elapsed: 3441.599335 sec
Has completed 383600 jobs
Time elapsed: 3441.877683 sec
Has completed 383700 jobs
Time elapsed: 3442.176433 sec
Has completed 383800 jobs
Time elapsed: 3442.467299 sec
Has completed 383900 jobs
Time elapsed: 3442.817104 sec
Has completed 384000 jobs
Time elapsed: 3443.134899 sec
Has completed 384100 jobs
Time elapsed: 3443.426686 sec
Has completed 384200 jobs
Time elapsed: 3443.766456 sec
Has completed 384300 jobs
Time elapsed: 3444.096754 sec
Has completed 384400 jobs
Time elapsed: 3444.419003 sec
Has completed 384500 jobs
Time elapsed: 3444.741440 sec
Has completed 384600 jobs
Time elapsed: 3445.063358 sec
Has completed 384700 jobs
Time elapsed: 3445.365245 sec
Has completed 384800 jobs
Time elapsed: 3445.726295 sec
Has completed 384900 jobs
Time elapsed: 3446.087605 sec
Has completed 385000 jobs
Time elapsed: 3446.435328 sec
Has completed 385100 jobs
Time elapsed: 3446.787

Has completed 398100 jobs
Time elapsed: 3488.705243 sec
Has completed 398200 jobs
Time elapsed: 3489.024670 sec
Has completed 398300 jobs
Time elapsed: 3489.356108 sec
Has completed 398400 jobs
Time elapsed: 3489.744740 sec
Has completed 398500 jobs
Time elapsed: 3490.075223 sec
Has completed 398600 jobs
Time elapsed: 3490.480947 sec
Has completed 398700 jobs
Time elapsed: 3490.955305 sec
Has completed 398800 jobs
Time elapsed: 3491.382544 sec
Has completed 398900 jobs
Time elapsed: 3491.754202 sec
Has completed 399000 jobs
Time elapsed: 3492.137276 sec
Has completed 399100 jobs
Time elapsed: 3492.502150 sec
Has completed 399200 jobs
Time elapsed: 3492.843389 sec
Has completed 399300 jobs
Time elapsed: 3493.166915 sec
Has completed 399400 jobs
Time elapsed: 3493.547874 sec
Has completed 399500 jobs
Time elapsed: 3493.894004 sec
Has completed 399600 jobs
Time elapsed: 3494.262483 sec
Has completed 399700 jobs
Time elapsed: 3494.589519 sec
Has completed 399800 jobs
Time elapsed: 3494.938

Has completed 412800 jobs
Time elapsed: 3536.378848 sec
Has completed 412900 jobs
Time elapsed: 3536.692495 sec
Has completed 413000 jobs
Time elapsed: 3537.013305 sec
Has completed 413100 jobs
Time elapsed: 3537.319215 sec
Has completed 413200 jobs
Time elapsed: 3537.608677 sec
Has completed 413300 jobs
Time elapsed: 3537.920854 sec
Has completed 413400 jobs
Time elapsed: 3538.233538 sec
Has completed 413500 jobs
Time elapsed: 3538.514927 sec
Has completed 413600 jobs
Time elapsed: 3538.802545 sec
Has completed 413700 jobs
Time elapsed: 3539.084566 sec
Has completed 413800 jobs
Time elapsed: 3539.355992 sec
Has completed 413900 jobs
Time elapsed: 3539.680799 sec
Has completed 414000 jobs
Time elapsed: 3540.026294 sec
Has completed 414100 jobs
Time elapsed: 3540.383819 sec
Has completed 414200 jobs
Time elapsed: 3540.709853 sec
Has completed 414300 jobs
Time elapsed: 3541.092113 sec
Has completed 414400 jobs
Time elapsed: 3541.434291 sec
Has completed 414500 jobs
Time elapsed: 3541.750

In [14]:
freq_counter.most_common(50)

[('行車', 4578),
 ('技巧', 2607),
 ('上線', 2290),
 ('接手', 1985),
 ('罰款', 1852),
 ('分成', 1757),
 ('電動車', 1537),
 ('電商', 1534),
 ('頂尖', 1356),
 ('專區', 1320),
 ('耳機', 1116),
 ('外帶', 1101),
 ('破局', 1097),
 ('視頻', 1080),
 ('經警', 1045),
 ('務實', 1024),
 ('加快', 1010),
 ('場均', 969),
 ('解放軍', 946),
 ('特首', 852),
 ('央視', 846),
 ('公安', 822),
 ('下海', 782),
 ('網絡', 779),
 ('聯網', 671),
 ('保安', 665),
 ('一次性', 652),
 ('下課', 624),
 ('網民', 586),
 ('匝道', 550),
 ('奔馳', 531),
 ('比特', 513),
 ('質量', 504),
 ('高校', 461),
 ('信息', 454),
 ('大專', 448),
 ('定點', 429),
 ('總書記', 399),
 ('旅遊局', 394),
 ('本體', 379),
 ('轉會', 367),
 ('看點', 363),
 ('麻豆', 359),
 ('互聯網', 341),
 ('筆記本', 341),
 ('被叫', 340),
 ('北漂', 334),
 ('說唱', 330),
 ('摩', 330),
 ('全家福', 320)]